# Driver Splits

Script to rebase the split times for a stage and display them relative to a specified driver.

In [98]:
import notebookimport

if __name__=='__main__':
    rebase='overallleader'
    rebase='PAD'#'PAD'
    MAXINSPLITDELTA=20 #set xlim on the within split delta
    ss='SS12'
    drivercode=rebase

In [99]:
sr = __import__("Charts - Stage Results")
ssd = __import__("Charts - Split Sector Delta")

In [100]:
import os
import sqlite3
import pandas as pd
import pytablewriter
import six

#dbname='wrc18.db'
#dbname='france18.db'
#conn = sqlite3.connect(dbname)

if __name__=='__main__':
    #dbname='wrc18.db'
    dbname='italy18.db'
    conn = sqlite3.connect(dbname)
    rally='Italy'
    rc='RC1'
    year=2018
    #ss='SS4'

In [101]:
stageresults = sr.dbGetRallyStages(conn, rally).sort_values('number')
stageresults.head()

,legDate,date,startListId,status,itineraryLegId,itinerarySectionId,section,order,code,distance,eventId,name,number,stageId,stageType,status,timingPrecision,itineraryLegId,itinerarySections.itinerarySectionId
0,2018-06-07,Thursday 7th June,153.0,Completed,92,198,Section 1,1,SS1,2.00,32,ITTIRI ARENA SHOW,1,500,HeadToHeadSuperSpecialStage,Completed,Tenth,92,198
1,2018-06-08,Friday 8th June,157.0,Running,91,199,Section 2,2,SS2,22.12,32,TULA 1,2,502,SpecialStage,Completed,Tenth,91,199
2,2018-06-08,Friday 8th June,157.0,Running,91,199,Section 2,2,SS3,14.37,32,CASTELSARDO 1,3,503,SpecialStage,Completed,Tenth,91,199
3,2018-06-08,Friday 8th June,157.0,Running,91,199,Section 2,2,SS4,14.14,32,TERGU - OSILO 1,4,505,SpecialStage,Completed,Tenth,91,199
4,2018-06-08,Friday 8th June,157.0,Running,91,199,Section 2,2,SS5,11.46,32,MONTE BARANTA 1,5,506,SpecialStage,Completed,Tenth,91,199


In [157]:
stagerank_overall.columns

Index(['diffFirst', 'diffFirstMs', 'diffPrev', 'diffPrevMs', 'entryId',
       'penaltyTime', 'penaltyTimeMs', 'position', 'stageTime', 'stageTimeMs',
       'totalTime', 'totalTimeMs', 'stageId', 'class', 'code', 'distance',
       'name', 'snum', 'driver.code', 'entrant.name', 'classrank',
       'gainedClassPos', 'gainedClassLead', 'classPosDiff', 'lostClassLead',
       'retainedClassLead', 'gainedOverallPos', 'gainedOverallLead',
       'overallPosDiff', 'lostOverallLead', 'retainedOverallLead', 'stagewin',
       'stagewincount', 'winsinarow', 'gainedTime'],
      dtype='object')

In [163]:
def rebaseOverallRallyTime(stagerank_overall, drivercode, code):
    rebase = stagerank_overall[stagerank_overall['driver.code']==drivercode][['code','totalTimeMs']].set_index('code').to_dict(orient='dict')['totalTimeMs']
    stagerank_overall['rebased'] = stagerank_overall['totalTimeMs'] - stagerank_overall['code'].map(rebase)
    return stagerank_overall

In [179]:
zz=rebaseOverallRallyTime(stagerank_overall, 'PAD', ss)
zz=zz[zz['code']==ss][['driver.code','position','totalTimeMs','code', 'rebased']].set_index('driver.code')
zz['Overall Time']=-zz['rebased']/1000
zz

,position,totalTimeMs,code,rebased,Overall Time
driver.code,,,,,
OGI,1,8323500,SS12,-70100,70.1
NEU,2,8328400,SS12,-65200,65.2
LAT,3,8364600,SS12,-29000,29.0
LAP,4,8371600,SS12,-22000,22.0
PAD,5,8393600,SS12,0,0.0
OST,6,8398500,SS12,4900,-4.9
BRE,7,8450100,SS12,56500,-56.5
TÄN,8,8949600,SS12,556000,-556.0
SUN,9,9148100,SS12,754500,-754.5


In [160]:
stagerank_overall = sr.getEnrichedStageRank(conn, rally, typ='overall')
stagerank_overall[stagerank_overall['code']==ss][['driver.code','position','totalTimeMs','code']]

,driver.code,position,totalTimeMs,code
164,OGI,1,8323500,SS12
165,NEU,2,8328400,SS12
166,LAT,3,8364600,SS12
167,LAP,4,8371600,SS12
168,PAD,5,8393600,SS12
169,OST,6,8398500,SS12
170,BRE,7,8450100,SS12
171,TÄN,8,8949600,SS12
172,SUN,9,9148100,SS12
173,EVA,10,9218100,SS12


In [151]:
stagerank_stage = sr.getEnrichedStageRank(conn, rally, typ='stage')
stagerank_stage[stagerank_stage['driver.code']=='PAD'][['position','code']]

,position,code
4,5.0,SS1
24,10.0,SS2
35,6.0,SS3
53,9.0,SS4
68,9.0,SS5
79,5.0,SS6
99,10.0,SS7
112,8.0,SS8
125,6.0,SS9
143,9.0,SS10


In [103]:
splits = ssd.dbGetSplits(conn,rally,ss,rc)

elapseddurations=ssd.getElapsedDurations(splits)
elapseddurations.head()

,drivercode,elapsedDurationS,startDateTime,section
0,EVA,214.3,2018-06-09T08:17:00,1
1,EVA,393.0,2018-06-09T08:17:00,2
2,EVA,559.1,2018-06-09T08:17:00,3
3,EVA,717.2,2018-06-09T08:17:00,4
4,EVA,917.9,2018-06-09T08:17:00,5


In [238]:
roadPos=splits[['drivercode','startDateTime']].drop_duplicates()
roadPos = roadPos.set_index('drivercode')
roadPos['Road Position']=roadPos['startDateTime'].rank().astype(int).astype(str)
roadPos

,startDateTime,Road Position
drivercode,,
EVA,2018-06-09T08:17:00,1
BRE,2018-06-09T08:20:00,2
PAD,2018-06-09T08:23:00,3
OST,2018-06-09T08:26:00,4
LAP,2018-06-09T08:29:00,5
LAT,2018-06-09T08:32:00,6
NEU,2018-06-09T08:35:00,7
OGI,2018-06-09T08:38:00,8
TÄN,2018-06-09T08:41:00,9


In [141]:
splits[splits['drivercode'].isin(['PAD','NEU'])]

,elapsedDuration,elapsedDurationMs,entryId,splitDateTime,splitDateTimeLocal,splitPointId,splitPointTimeId,stageTimeDuration,stageTimeDurationMs,startDateTime,startDateTimeLocal,stageId,class,code,distance,name,drivercode,elapsedDurationS
10,PT3M32.518S,212518,2280,2018-06-09T08:26:32.518Z,2018-06-09T10:26:32.518+02:00,1261,34743,00:18:06.6000000,1086600.0,2018-06-09T08:23:00,2018-06-09T10:23:00+02:00,517,RC1,SS12,28.89,MONTE LERNO 1,PAD,212.5
11,PT6M28.231S,388231,2280,2018-06-09T08:29:28.231Z,2018-06-09T10:29:28.231+02:00,1264,34758,00:18:06.6000000,1086600.0,2018-06-09T08:23:00,2018-06-09T10:23:00+02:00,517,RC1,SS12,28.89,MONTE LERNO 1,PAD,388.2
12,PT9M9.603S,549603,2280,2018-06-09T08:32:09.603Z,2018-06-09T10:32:09.603+02:00,1267,34766,00:18:06.6000000,1086600.0,2018-06-09T08:23:00,2018-06-09T10:23:00+02:00,517,RC1,SS12,28.89,MONTE LERNO 1,PAD,549.6
13,PT11M44.023S,704023,2280,2018-06-09T08:34:44.023Z,2018-06-09T10:34:44.023+02:00,1263,34776,00:18:06.6000000,1086600.0,2018-06-09T08:23:00,2018-06-09T10:23:00+02:00,517,RC1,SS12,28.89,MONTE LERNO 1,PAD,704.0
14,PT14M59.772S,899772,2280,2018-06-09T08:37:59.772Z,2018-06-09T10:37:59.772+02:00,1259,34794,00:18:06.6000000,1086600.0,2018-06-09T08:23:00,2018-06-09T10:23:00+02:00,517,RC1,SS12,28.89,MONTE LERNO 1,PAD,899.8
30,PT3M31.928S,211928,2279,2018-06-09T08:38:31.928Z,2018-06-09T10:38:31.928+02:00,1261,34796,00:17:59.1000000,1079100.0,2018-06-09T08:35:00,2018-06-09T10:35:00+02:00,517,RC1,SS12,28.89,MONTE LERNO 1,NEU,211.9
31,PT6M27.455S,387455,2279,2018-06-09T08:41:27.455Z,2018-06-09T10:41:27.455+02:00,1264,34808,00:17:59.1000000,1079100.0,2018-06-09T08:35:00,2018-06-09T10:35:00+02:00,517,RC1,SS12,28.89,MONTE LERNO 1,NEU,387.5
32,PT9M8.593S,548593,2279,2018-06-09T08:44:08.593Z,2018-06-09T10:44:08.593+02:00,1267,34821,00:17:59.1000000,1079100.0,2018-06-09T08:35:00,2018-06-09T10:35:00+02:00,517,RC1,SS12,28.89,MONTE LERNO 1,NEU,548.6
33,PT11M42.685S,702685,2279,2018-06-09T08:46:42.685Z,2018-06-09T10:46:42.685+02:00,1263,34828,00:17:59.1000000,1079100.0,2018-06-09T08:35:00,2018-06-09T10:35:00+02:00,517,RC1,SS12,28.89,MONTE LERNO 1,NEU,702.7
34,PT14M55.209S,895209,2279,2018-06-09T08:49:55.209Z,2018-06-09T10:49:55.209+02:00,1259,34840,00:17:59.1000000,1079100.0,2018-06-09T08:35:00,2018-06-09T10:35:00+02:00,517,RC1,SS12,28.89,MONTE LERNO 1,NEU,895.2


In [104]:
rebasedelapseddurations = ssd.rebaseElapsedDurations(elapseddurations, drivercode)
rebasedelapseddurations.head()

,drivercode,elapsedDurationS,startDateTime,section,rebased
0,EVA,214.3,2018-06-09T08:17:00,1,1.8
1,EVA,393.0,2018-06-09T08:17:00,2,4.8
2,EVA,559.1,2018-06-09T08:17:00,3,9.5
3,EVA,717.2,2018-06-09T08:17:00,4,13.2
4,EVA,917.9,2018-06-09T08:17:00,5,18.1


In [114]:
rbe=-rebasedelapseddurations.pivot('drivercode','section','rebased')
rbe.columns=list(rbe.columns)[:-1]+['{} Overall'.format(ss)]
rbe=rbe.sort_values(rbe.columns[-1],ascending =False)
rbe

,1,2,3,4,5,SS12 Overall
drivercode,,,,,,
NEU,0.6,0.7,1.0,1.3,4.6,7.5
LAT,-1.1,-0.9,-1.0,-1.1,-0.6,0.8
PAD,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0
LAP,-0.3,-2.5,-3.4,-3.4,-2.3,-0.2
OST,-0.8,-2.2,-3.3,-4.0,-4.8,-5.1
OGI,-2.3,-4.7,-6.7,-8.3,-6.2,-7.1
TÄN,-1.6,-3.6,-5.5,-7.3,-7.7,-8.2
BRE,-2.7,-5.8,-9.9,-12.0,-15.1,-16.9
EVA,-1.8,-4.8,-9.5,-13.2,-18.1,-24.9


In [213]:
#https://pandas.pydata.org/pandas-docs/stable/style.html
def color_negative(val):
    """
    Takes a scalar and returns a string with
    the css property `'color: red'` for negative
    strings, black otherwise.
    """
    if isinstance(val, str): return ''
    elif val and (isinstance(val,int) or isinstance(val,float)):
        color = 'green' if val < 0 else 'red' if val > 0  else 'black'
    else:
        color='white'
    return 'color: %s' % color

In [214]:
s = rbe.style.applymap(color_negative)
s

,1,2,3,4,5,SS12 Overall
drivercode,,,,,,
NEU,0.6,0.7,1,1.3,4.6,7.5
LAT,-1.1,-0.9,-1,-1.1,-0.6,0.8
PAD,-0,-0,-0,-0,-0,-0
LAP,-0.3,-2.5,-3.4,-3.4,-2.3,-0.2
OST,-0.8,-2.2,-3.3,-4,-4.8,-5.1
OGI,-2.3,-4.7,-6.7,-8.3,-6.2,-7.1
TÄN,-1.6,-3.6,-5.5,-7.3,-7.7,-8.2
BRE,-2.7,-5.8,-9.9,-12,-15.1,-16.9
EVA,-1.8,-4.8,-9.5,-13.2,-18.1,-24.9


In [117]:
splitdurations = ssd.getSplitDurationsFromSplits(conn,rally,ss,rc)
splitdurations.head()

,drivercode,splitDurationS,startDateTime,stageTimeDurationMs,section
0,EVA,214.3,2018-06-09T08:17:00,1111500.0,1
1,EVA,178.7,2018-06-09T08:17:00,1111500.0,2
2,EVA,166.0,2018-06-09T08:17:00,1111500.0,3
3,EVA,158.1,2018-06-09T08:17:00,1111500.0,4
4,EVA,200.7,2018-06-09T08:17:00,1111500.0,5


In [118]:
rebasedSplits = ssd.rebaseSplitDurations(splitdurations, drivercode)
rebasedSplits.head()

,drivercode,splitDurationS,startDateTime,stageTimeDurationMs,section,rebased
0,EVA,214.3,2018-06-09T08:17:00,1111500.0,1,1.8
1,EVA,178.7,2018-06-09T08:17:00,1111500.0,2,3.0
2,EVA,166.0,2018-06-09T08:17:00,1111500.0,3,4.6
3,EVA,158.1,2018-06-09T08:17:00,1111500.0,4,3.7
4,EVA,200.7,2018-06-09T08:17:00,1111500.0,5,5.0


In [138]:
splitdurations[splitdurations['drivercode'].isin( ['PAD','NEU'])]

,drivercode,splitDurationS,startDateTime,stageTimeDurationMs,section,rebased
10,PAD,212.5,2018-06-09T08:23:00,1086600.0,1,0.0
11,PAD,175.7,2018-06-09T08:23:00,1086600.0,2,0.0
12,PAD,161.4,2018-06-09T08:23:00,1086600.0,3,0.0
13,PAD,154.4,2018-06-09T08:23:00,1086600.0,4,0.0
14,PAD,195.7,2018-06-09T08:23:00,1086600.0,5,0.0
69,PAD,186.8,2018-06-09T08:23:00,1086600.0,6,0.0
30,NEU,211.9,2018-06-09T08:35:00,1079100.0,1,-0.6
31,NEU,175.5,2018-06-09T08:35:00,1079100.0,2,-0.2
32,NEU,161.1,2018-06-09T08:35:00,1079100.0,3,-0.3
33,NEU,154.1,2018-06-09T08:35:00,1079100.0,4,-0.3


In [119]:
rbp=-rebasedSplits.pivot('drivercode','section','rebased')
rbp.columns=['D{}'.format(c) for c in rbp.columns]
rbp.sort_values(rbp.columns[-1],ascending =True)

,D1,D2,D3,D4,D5,D6
drivercode,,,,,,
EVA,-1.8,-3.0,-4.6,-3.7,-5.0,-6.8
SUN,-1.7,-3.0,-30.3,-139.2,-4.3,-5.1
BRE,-2.7,-3.1,-4.1,-2.1,-3.2,-1.8
OGI,-2.3,-2.4,-2.0,-1.5,2.0,-0.9
TÄN,-1.6,-2.0,-1.9,-1.8,-0.4,-0.5
OST,-0.8,-1.4,-1.0,-0.8,-0.9,-0.3
PAD,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0
LAT,-1.1,0.2,-0.1,-0.1,0.5,1.4
LAP,-0.3,-2.1,-0.9,-0.0,0.9,2.1


In [243]:
roadPos

,startDateTime,Road Position
drivercode,,
EVA,2018-06-09T08:17:00,1
BRE,2018-06-09T08:20:00,2
PAD,2018-06-09T08:23:00,3
OST,2018-06-09T08:26:00,4
LAP,2018-06-09T08:29:00,5
LAT,2018-06-09T08:32:00,6
NEU,2018-06-09T08:35:00,7
OGI,2018-06-09T08:38:00,8
TÄN,2018-06-09T08:41:00,9


In [248]:
rb2=pd.merge(rbe,rbp,left_index=True, right_index=True)
rb2=pd.merge(rb2,zz[['position','Overall Time']],left_index=True, right_index=True)
rb2=pd.merge(rb2,roadPos[['Road Position']],left_index=True, right_index=True)
cols=rb2.columns.tolist()
rb2=rb2[[cols[-1]]+cols[:-1]]
rb2

,Road Position,1,2,3,4,5,SS12 Overall,D1,D2,D3,D4,D5,D6,position,Overall Time
NEU,7,0.6,0.7,1.0,1.3,4.6,7.5,0.6,0.2,0.3,0.3,3.2,2.9,2,65.2
LAT,6,-1.1,-0.9,-1.0,-1.1,-0.6,0.8,-1.1,0.2,-0.1,-0.1,0.5,1.4,3,29.0
PAD,3,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,5,0.0
LAP,5,-0.3,-2.5,-3.4,-3.4,-2.3,-0.2,-0.3,-2.1,-0.9,-0.0,0.9,2.1,4,22.0
OST,4,-0.8,-2.2,-3.3,-4.0,-4.8,-5.1,-0.8,-1.4,-1.0,-0.8,-0.9,-0.3,6,-4.9
OGI,8,-2.3,-4.7,-6.7,-8.3,-6.2,-7.1,-2.3,-2.4,-2.0,-1.5,2.0,-0.9,1,70.1
TÄN,9,-1.6,-3.6,-5.5,-7.3,-7.7,-8.2,-1.6,-2.0,-1.9,-1.8,-0.4,-0.5,8,-556.0
BRE,2,-2.7,-5.8,-9.9,-12.0,-15.1,-16.9,-2.7,-3.1,-4.1,-2.1,-3.2,-1.8,7,-56.5
EVA,1,-1.8,-4.8,-9.5,-13.2,-18.1,-24.9,-1.8,-3.0,-4.6,-3.7,-5.0,-6.8,10,-824.5
SUN,10,-1.7,-4.7,-35.0,-174.2,-178.5,-183.6,-1.7,-3.0,-30.3,-139.2,-4.3,-5.1,9,-754.5


In [249]:
#There seems to be missing tenths?
#Elapsed durations are provided in milliseconds. Need to round correctly to tenths?
#Elapsed times grabbed from ssd.dbGetSplits(conn,rally,ss,rc)
from numpy import NaN
rb2=rb2.replace(0,NaN)
rb2=rb2.fillna('')
rb2['position']=rb2['position'].astype(str)
rb2.rename(columns={'position': 'Overall Position'}, inplace=True)
s=rb2.style.applymap(color_negative)
#data.style.applymap(highlight_cols, subset=pd.IndexSlice[:, ['B', 'C']])

s.set_caption("{}: running split times and deltas within each split.".format(ss))


,Road Position,1,2,3,4,5,SS12 Overall,D1,D2,D3,D4,D5,D6,Overall Position,Overall Time
NEU,7,0.6,0.7,1,1.3,4.6,7.5,0.6,0.2,0.3,0.3,3.2,2.9,2,65.2
LAT,6,-1.1,-0.9,-1,-1.1,-0.6,0.8,-1.1,0.2,-0.1,-0.1,0.5,1.4,3,29
PAD,3,,,,,,,,,,,,,5,
LAP,5,-0.3,-2.5,-3.4,-3.4,-2.3,-0.2,-0.3,-2.1,-0.9,,0.9,2.1,4,22
OST,4,-0.8,-2.2,-3.3,-4,-4.8,-5.1,-0.8,-1.4,-1,-0.8,-0.9,-0.3,6,-4.9
OGI,8,-2.3,-4.7,-6.7,-8.3,-6.2,-7.1,-2.3,-2.4,-2,-1.5,2,-0.9,1,70.1
TÄN,9,-1.6,-3.6,-5.5,-7.3,-7.7,-8.2,-1.6,-2,-1.9,-1.8,-0.4,-0.5,8,-556
BRE,2,-2.7,-5.8,-9.9,-12,-15.1,-16.9,-2.7,-3.1,-4.1,-2.1,-3.2,-1.8,7,-56.5
EVA,1,-1.8,-4.8,-9.5,-13.2,-18.1,-24.9,-1.8,-3,-4.6,-3.7,-5,-6.8,10,-824.5
SUN,10,-1.7,-4.7,-35,-174.2,-178.5,-183.6,-1.7,-3,-30.3,-139.2,-4.3,-5.1,9,-754.5


In [122]:
from IPython.display import HTML

styles = [
    dict(selector="caption", props=[("caption-side", "bottom")])
]
html = (df.style.set_table_styles(styles)
          .set_caption("Hover to highlight."))
html

NameError: name 'df' is not defined